# Example to train a model
Using the openstf tasks

In [47]:
import pandas as pd
import IPython
from openstf.pipeline.train_model import train_model_pipeline
from openstf.pipeline.create_forecast import create_forecast_pipeline

In [68]:
# define properties of training/prediction. We call this a 'prediction_job'
pj=dict(id=287,
        model='xgb',
        quantiles=[10,30,50,70,90],
        hyper_params={}, # Note, this should become optional
        feature_names=None, # Note, this should become optional
       )

# Load input data
input_data = pd.read_csv('data/get_model_input_pid_287.csv', index_col='index', parse_dates=True)

# Split in training and forecasting data
train_data = input_data.iloc[:-200,:] # everything except last 200 rows (~ 48 hours)
to_forecast_data = input_data.iloc[:-200,:] # last 200 rows


In [40]:
train_data.head()

,load,APX,clouds,radiation,temp,winddeg,windspeed,windspeed_100m,pressure,humidity,...,sjv_E1A,sjv_E1B,sjv_E1C,sjv_E2A,sjv_E2B,sjv_E3A,sjv_E3B,sjv_E3C,sjv_E3D,sjv_E4A
index,,,,,,,,,,,,,,,,,,,,,
2020-10-02 09:45:00+00:00,3.310000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02 10:00:00+00:00,2.620000,34.0,99.758911,1.552899e+06,16.449036,154.711456,3.527778,9.349441,99453.476562,0.686240,...,0.000031,0.000030,0.000029,0.000033,0.000032,0.000061,0.000048,0.000048,0.000031,0.0
2020-10-02 10:15:00+00:00,0.796667,34.0,99.819193,1.575618e+06,16.400948,157.491554,3.557639,9.232026,99416.363281,0.683780,...,0.000032,0.000030,0.000029,0.000033,0.000032,0.000060,0.000048,0.000048,0.000031,0.0
2020-10-02 10:30:00+00:00,0.300000,34.0,99.879475,1.598338e+06,16.352859,160.271652,3.587500,9.114612,99379.250000,0.681319,...,0.000032,0.000031,0.000029,0.000033,0.000031,0.000058,0.000048,0.000048,0.000031,0.0
2020-10-02 10:45:00+00:00,1.773333,34.0,99.939756,1.594736e+06,16.304771,163.051750,3.617361,8.997197,99342.136719,0.678859,...,0.000032,0.000030,0.000029,0.000032,0.000031,0.000057,0.000048,0.000048,0.000031,0.0


In [41]:
to_forecast_data.head()

,APX,clouds,radiation,temp,winddeg,windspeed,windspeed_100m,pressure,humidity,rain,...,sjv_E1A,sjv_E1B,sjv_E1C,sjv_E2A,sjv_E2B,sjv_E3A,sjv_E3B,sjv_E3C,sjv_E3D,sjv_E4A
index,,,,,,,,,,,,,,,,,,,,,
2020-10-02 09:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02 10:00:00+00:00,34.0,99.758911,1.552899e+06,16.449036,154.711456,3.527778,9.349441,99453.476562,0.686240,0.000,...,0.000031,0.000030,0.000029,0.000033,0.000032,0.000061,0.000048,0.000048,0.000031,0.0
2020-10-02 10:15:00+00:00,34.0,99.819193,1.575618e+06,16.400948,157.491554,3.557639,9.232026,99416.363281,0.683780,0.005,...,0.000032,0.000030,0.000029,0.000033,0.000032,0.000060,0.000048,0.000048,0.000031,0.0
2020-10-02 10:30:00+00:00,34.0,99.879475,1.598338e+06,16.352859,160.271652,3.587500,9.114612,99379.250000,0.681319,0.010,...,0.000032,0.000031,0.000029,0.000033,0.000031,0.000058,0.000048,0.000048,0.000031,0.0
2020-10-02 10:45:00+00:00,34.0,99.939756,1.594736e+06,16.304771,163.051750,3.617361,8.997197,99342.136719,0.678859,0.015,...,0.000032,0.000030,0.000029,0.000032,0.000031,0.000057,0.000048,0.000048,0.000031,0.0


# Train a model
Train a model using the high-level pipeline. Store the model and reports on training proces in ./trained_models

In [45]:
train_model_pipeline(
    pj,
    train_data,
    check_old_model_age=False,
    trained_models_folder='./trained_models',
    save_figures_folder=f"./trained_models/{pj['id']}",
    )

2021-08-25 10:44.24 [debug    ] Trying to load model from: trained_models\287\20210825083641\model.joblib
2021-08-25 10:44.25 [debug    ] Changed 2 values of constant load to NA. num_const_load_values=2
2021-08-25 10:44.25 [debug    ] Removed 2 NA values            num_removed_values=2
2021-08-25 10:44.30 [info     ] New model is better than old model, continuing with training procces
2021-08-25 10:44.32 [info     ] Saved model to trained_models\287\20210825084432\model.joblib
2021-08-25 10:44.34 [info     ] Saved figures to trained_models\287


You can find the trained model in ./trained_models, along with reports on the training process

In [66]:
## Inspect local files
IPython.display.HTML(f"<iframe src=./trained_models/{pj['id']}/Predictor0.25.html width=800 height=400></iframe>"
                     f"<iframe src=./trained_models/{pj['id']}/Predictor47.0.html width=800 height=400></iframe>"
                     f"<iframe src=./trained_models/{pj['id']}/weight_plot.html width=800 height=400></iframe>")